#Automatizált vizsgalap feldolgozás

A beadandó feladatnak egy meghatározott formájú teszt vizsgalap automatikus feldolgozását választottam.
A vizsgalapot a feldolgozást megkönnyítendő, valamint a hibalehetőségek csökkentése érdekében kitalált formátumú. A teszt kérdései és válaszai, valamint azok sorrendje minden vizsgázó esetére automatikus generálható  megfelelő adatbázisból.

A vizsgalap fejlécén a tárgy neve, a vizsgázó neve, Neptun kódja, valamint ezen adatokat tartalmazó QR kód található, mely utóbbi kódolva tartalmazza a vizsgalap kérdéseire adott helyes válaszokat az automatikus kiértékeléshez.
(Az feladat kidolgozása során használt vizsgalap nem a fent leírt módon generált, csak Word állományban került összállításra)

<img src="./img/vizsgalap.png" width=500/>

A feladat kidolgozása Python nyelven, több függvénykönyvtár felhasználásával (többek között az OpenCV) készült.

In [8]:
#Függvénykönyvtárak importálása
import numpy as np
import cv2
from pyzbar.pyzbar import decode

In [9]:
#A feldolgozandó képek beolvasása
path = './Kepek/'
im = cv2.imread(path+'Kepolvasas_20181119_123603_002.tif')

A kép apró zajainak (só/bors) szűrésére elmosást (blur) alkalmazok,
majd a képet szürkeárnyalatossá konvertálom, küszöböléssel bináris (fekete-fehér) képpé alakítom, 
végül kontúrkeresés futtatok hierarchia-fa felépítéssel.

In [10]:
im2 = cv2.blur(im,(3,3))
imgray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)

ret, thresh = cv2.threshold(imgray, 55, 255, cv2.THRESH_OTSU)
_, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)


A megtalált kontúrok közül hierarchiájában megkeressük amelyiknek az 1. számú a szülője, 
melyet egy, az Y koordinátával indexelt asszociatív tömbbel (dictionary) tárolunk. 
(Így a vizsgalap legkülső keretén belüli elválasztott tartományokat kapjuk meg.)

In [11]:
elso_szint_dict = {}

for i in range(0, len(contours)):
    if hierarchy[0,i,3] == 1:
        elso_szint_dict[contours[i][0][0][1]] = i

A fenti tömb értékeit felhasználva megkeressük azok gyerek régióit, melyek 4-4 alrégiót tartalmaznak.
(Ezzel a válaszok megjelölésére használt négyzetek körüli keret tartományt kapjuk.)

In [12]:
jel_kontener_dict = {}

for h in elso_szint_dict.values(): 
    alkontur=0
    for i in range(0, len(contours)):
         if hierarchy[0,i,3] == h:
            alkontur+=1
    if alkontur==4:
        jel_kontener_dict[contours[h][0][0][1]] = h

Végül a kontúrlistából kiválogatjuk azokat, melyek a fenti régióba esnek, azaz a jelölőnégyzeteket.


In [14]:
negyzetek_dict = {}

for b in jel_kontener_dict.values():
    for i in range(0, len(contours)):
         if hierarchy[0,i,3] == b:
             negyzetek_dict[contours[i][0][0][1]] = i


A lenti kód végigmegy a jelölőnégyzet régiók listáján, és megvizsgálja, hogy azok befoglaló négyzetén belül
mennyi a fehér és fekete képpontok száma. A vizsgálathoz nem a befoglaló négyzet hanem egy attól 3-3 képponttal kisebb négyzeten belüli területet vizsgál. Ez azért szükséges, hogy a jelölőnégyzet fekete keretének képpontjai  ne kerüljenek megszámlálásre.
A fekete képpontok arányát hasonlítva az összes (fekete plusz fehér) képpontok számához kapjuk hogy az adott jelölőnégyzet megjelölésre került-e a vizsgázó által.

A kód egy gyors listát ír ki a kérdésekhez kapcsolódó jelölőnégyzetek bejelöltségéről.
(Ezen kódot alapul véve, és a lenti QR kód feldolgozás alapján - az abban kódolt helyes válasz adatok segítségével - lehet egy valódi kiértékelő algoritmust készíteni, mely a helyes válaszok alapján számít pontokat, összpontszámot és végül osztályzatot, de ez már nem a képfeldolgozás témaköre)

In [15]:
kerdes_szam = 1

k_dict= {1:"a", 2: "b", 3:"c", 4:"d"}

for d in sorted(jel_kontener_dict.keys()):
    print( str(kerdes_szam) + '. kerdes:')
    valasz_szam=1
    for k in sorted(negyzetek_dict):
        if hierarchy[0,negyzetek_dict[k],3] == jel_kontener_dict[d]:
            x,y,w,h = cv2.boundingRect(contours[negyzetek_dict[k]])
            black = 0
            white = 0
            rx = range(x+3, x+w-3)        #mivel a befoglaló négyzet a 
            ry = range(y+3, y+h-3)
            for x1 in rx:
                for y1 in ry:
                    pont = thresh[y1][x1]
                    if  pont < 100: 
                        black+=1
                    else:
                         white+=1

            if black/(black+white) >0.5 :
                print(k_dict[valasz_szam] + '. valasz bejelolve' )
            else:
                print(k_dict[valasz_szam] + '. valasz nincs bejelolve' )
            valasz_szam+=1
    kerdes_szam+=1

1. kerdes:
a. valasz nincs bejelolve
b. valasz nincs bejelolve
c. valasz nincs bejelolve
d. valasz nincs bejelolve
2. kerdes:
a. valasz bejelolve
b. valasz nincs bejelolve
c. valasz nincs bejelolve
d. valasz nincs bejelolve
3. kerdes:
a. valasz nincs bejelolve
b. valasz nincs bejelolve
c. valasz nincs bejelolve
d. valasz nincs bejelolve
4. kerdes:
a. valasz nincs bejelolve
b. valasz bejelolve
c. valasz nincs bejelolve
d. valasz nincs bejelolve
5. kerdes:
a. valasz bejelolve
b. valasz nincs bejelolve
c. valasz nincs bejelolve
d. valasz nincs bejelolve


Végül a vizsgalap fejléc QR kódjának adatainak kinyerése történik, a vizsgázó nevének és Neptun kódjának kinyerése, illetve a QR kódba lévő, vizsgalap kiértékelést segítő, helyes válaszok adatát kódoló karaktersorozat kiíratása.

In [16]:
qr_data = decode(im)[0][0].decode().split(';')
print('Név: ' + qr_data[0] + ', Neptun kód: ' + qr_data[1])
print('Kérdés-válaszok adatkódja: ' + qr_data[2])

Név: Egri Sandor Attila, Neptun kód:  NEPTUN-KOD
Kérdés-válaszok adatkódja:  7541-be76-38b4-44b4-a3a7-f015-5c38-ba59
